<a href="https://colab.research.google.com/github/dino-ali/olamor/blob/main/ModeloProyectoOptimizacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ortools

In [ ]:
# Primero, cargamos los datos del archivo Excel.
import pandas as pd
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)
df = pd.read_excel("/content/drive/MyDrive/Proyecto optimización./opti.xlsx")

# Convertimos las horas a minutos para facilitar los cálculos.
df['Available_Time_Minutes'] = df['Available_Time'].dt.hour * 60 + df['Available_Time'].dt.minute
df['Deadline_Minutes'] = df['Deadline'].dt.hour * 60 + df['Deadline'].dt.minute

# Utilizamos CBC para resolver este problema.
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver('CBC')

# Verificamos que el solver se haya creado correctamente.
if not solver:
    print('No se puede crear el solver. ¡Atención requerida!')
    exit(1)

# Ahora, definimos las variables y restricciones del problema.
num_articles = len(df)
num_trucks = len(df['Area'].unique())
x = {}  # x[i, k]: 1 si el artículo i se asigna al camión k, 0 de lo contrario
T = {}  # T[k]: Hora de inicio del camión k
t = {}  # t[i, k]: Hora de llegada del artículo i asignado al camión k
s = {}  # s[i, k]: Tiempo de permanencia del artículo i en el camión k

# Definimos las variables de decisión necesarias para el problema.
for i in range(num_articles):
    for k in range(num_trucks):
        x[(i, k)] = solver.BoolVar(f'x[{i},{k}]')  # Variable binaria (0 o 1)
        t[(i, k)] = solver.NumVar(0, solver.infinity(), f't[{i},{k}]')  # Tiempo de llegada del artículo i al camión k
        s[(i, k)] = solver.NumVar(0, solver.infinity(), f's[{i},{k}]')  # Tiempo de permanencia del artículo i en el camión k

# Establecemos las reglas del problema.
for k, area in enumerate(df['Area'].unique()):
    T[k] = solver.NumVar(0, solver.infinity(), f'T[{k}]')  # Hora de inicio del camión k
    capacidad_area = df[df['Area'] == area]['Area'].values[0]
    capacidad_peso = df[df['Area'] == area]['Weight'].values[0]

    # Restricción de capacidad de área del camión k
    solver.Add(solver.Sum(df['Area'][i] * x[(i, k)] for i in range(num_articles)) <= capacidad_area)

    # Restricción de capacidad de peso del camión k
    solver.Add(solver.Sum(df['Weight'][i] * x[(i, k)] for i in range(num_articles)) <= capacidad_peso)

# Aseguramos el cumplimiento de los plazos de entrega.
for i in range(num_articles):
    deadline = df['Deadline_Minutes'][i]
    for k in range(num_trucks):
        solver.Add(t[(i, k)] <= deadline * x[(i, k)])

# Evitamos que ciertos artículos vayan juntos en el mismo camión.
for k in range(num_trucks):
    solver.Add(x[(0, k)] + x[(1, k)] <= 1)

# Garantizamos que los artículos no pasen demasiado tiempo en los camiones.
for i in range(num_articles):
    for k in range(num_trucks):
        solver.Add(t[(i, k)] + s[(i, k)] <= df['Deadline_Minutes'][i] * x[(i, k)])

# Definimos la función objetivo: minimizar el tiempo total de entrega.
solver.Minimize(solver.Sum(t[(i, k)] for i in range(num_articles) for k in range(num_trucks)))

# Establecemos un límite de tiempo para resolver este problema.
solver.set_time_limit(600000)

# Resolvemos el problema.
status = solver.Solve()

# Analizamos los resultados.
if status == pywraplp.Solver.OPTIMAL:
    print('¡Se ha encontrado una solución óptima! Es la siguiente:')
    # Imprimimos la asignación de artículos a camiones y los tiempos correspondientes.
    for i in range(num_articles):
        for k in range(num_trucks):
            if x[(i, k)].solution_value() > 0:
                print(f'Artículo {df["Item_ID"][i]} se asigna al camión {k}')
                print(f'El camión {k} inicia su ruta a las {T[k].solution_value()} minutos')
                print(f'El artículo llega a su destino a los {t[(i, k)].solution_value()} minutos')
elif status == pywraplp.Solver.FEASIBLE:
    print('¡Se ha encontrado una solución factible!')
    # Imprimimos la asignación de artículos a camiones y los tiempos correspondientes.
else:
    print('No se ha encontrado una solución óptima ni factible. Se requiere atención.')


Mounted at /content/drive
¡Se ha encontrado una solución óptima! Es la siguiente:
